# Конфигурация

In [1]:
from pprint import pprint
from copy import deepcopy
import time

import pandas as pd
import numpy as np

from tqdm.auto import tqdm

import rectools 
from rectools import Columns
from rectools.dataset import Interactions, Dataset
from rectools.models import RandomModel, PopularModel
from rectools.model_selection import TimeRangeSplitter
from rectools.metrics import MAP, NDCG, Precision, Recall, MeanInvUserFreq, Serendipity, calc_metrics
from rectools.model_selection import TimeRangeSplitter

In [2]:
K_RECOS = 10
RANDOM_SEED = 32

np.random.seed(RANDOM_SEED)

# Загрузка данных

In [3]:
def headtail(df):
    return pd.concat([df.head(), df.tail()])

In [4]:
interactions = pd.read_csv('data_original/interactions.csv', parse_dates=["last_watch_dt"])

In [5]:
interactions.rename(
    columns={
        'last_watch_dt': Columns.Datetime,
        'total_dur': Columns.Weight
    }, 
    inplace=True)

interactions = Interactions(interactions)

In [6]:
headtail(interactions.df)

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
2,656683,7107,2021-05-09,10.0,0.0
3,864613,7638,2021-07-05,14483.0,100.0
4,964868,9506,2021-04-30,6725.0,100.0
5476246,648596,12225,2021-08-13,76.0,0.0
5476247,546862,9673,2021-04-13,2308.0,49.0
5476248,697262,15297,2021-08-20,18307.0,63.0
5476249,384202,16197,2021-04-19,6203.0,100.0
5476250,319709,4436,2021-08-15,3921.0,45.0


In [7]:
interactions.df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5476251 entries, 0 to 5476250
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   user_id      int64         
 1   item_id      int64         
 2   datetime     datetime64[ns]
 3   weight       float64       
 4   watched_pct  float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 208.9 MB


In [8]:
users = pd.read_csv('data_original/users.csv')
items = pd.read_csv('data_original/items.csv')

# Инициализация моделей, метрик, сплиттера

Согласно заданию в качестве моделей используется рандомная и популярная модель

In [9]:
models = {'random': RandomModel(random_state=32),
          'popular': PopularModel()}

В качестве метрик используются:

2 ранжирующие -- MAP, NDCG

2 классификационные -- Precision, Recall

2 beyond-accuracy -- MIUF (Novelty), Serendipity

In [10]:
metrics = {'MAP@1': MAP(k=1), 'MAP@5': MAP(k=5), 'MAP@10': MAP(k=10),
           'NDCG@1': NDCG(k=1), 'NDCG@5': NDCG(k=5), 'NDCG@10': NDCG(k=10),
           'Precision@1': Precision(k=1), 'Precision@5': Precision(k=5), 'Precision@10': Precision(k=10),
           'Recall@1': Recall(k=1), 'Recall@5': Recall(k=5), 'Recall@10': Recall(k=10),
           'MeanInvUserFreq@1': MeanInvUserFreq(k=1), 'MeanInvUserFreq@5': MeanInvUserFreq(k=5), 'MeanInvUserFreq@10': MeanInvUserFreq(k=10),
           'Serendipity@1': Serendipity(k=1), 'Serendipity@5': Serendipity(k=5), 'Serendipity@10': Serendipity(k=10)}

Согласно заданию используется сплиттер на 3 фолда для кросс-валидации по неделе c исключением холодных юзеров, айтемов и просмотренных айтемов

In [11]:
splitter = TimeRangeSplitter(test_size="7D",
                             n_splits=3,
                             filter_cold_users = True,
                             filter_cold_items= True,
                             filter_already_seen = True)

# Проведение обучения и валидации с расчётом метрик

In [12]:
def computation_metrics(models: dict, metrics: dict, splitter, K: int):

    results = []

    fold_iterator = splitter.split(interactions, collect_fold_stats=True)

    for train_ids, test_ids, fold_info in tqdm((fold_iterator), total=splitter.n_splits):
        print(f"\n{'='*20} Fold {fold_info['i_split']} {'='*20}")
        pprint(fold_info)

        # создаём train часть
        df_train = interactions.df.iloc[train_ids]
        dataset = Dataset.construct(df_train)

        # создаём test часть
        df_test = interactions.df.iloc[test_ids][Columns.UserItem]
        test_users = np.unique(df_test[Columns.User])
    
        catalog = df_train[Columns.Item].unique()
    
        for model_name, model in models.items():
            # инициализируем модель
            model = deepcopy(model)

            # обучаем модель
            start = time.time()
            model.fit(dataset)
            stop = time.time()

            # получаем рекомендации
            recos = model.recommend(
                users=test_users,
                dataset=dataset,
                k=K,
                filter_viewed=True,
            )

            # считаем метрики
            metric_values = calc_metrics(
                metrics,
                reco=recos,
                interactions=df_test,
                prev_interactions=df_train,
                catalog=catalog,
            )
            
            res = {"fold": fold_info["i_split"],
                   "model": model_name,
                   "train_time": stop - start}
            res.update(metric_values)
            
            results.append(res)
    
    return results

In [13]:
results = computation_metrics(models, metrics, splitter, K_RECOS)

  0%|          | 0/3 [00:00<?, ?it/s]


==================== Fold 0 ====================
{'end': Timestamp('2021-08-09 00:00:00', freq='7D'),
 'i_split': 0,
 'start': Timestamp('2021-08-02 00:00:00', freq='7D'),
 'test': 263681,
 'test_items': 6602,
 'test_users': 98184,
 'train': 4266013,
 'train_items': 15237,
 'train_users': 797423}

==================== Fold 1 ====================
{'end': Timestamp('2021-08-16 00:00:00', freq='7D'),
 'i_split': 1,
 'start': Timestamp('2021-08-09 00:00:00', freq='7D'),
 'test': 279422,
 'test_items': 6698,
 'test_users': 103511,
 'train': 4649162,
 'train_items': 15415,
 'train_users': 850489}

==================== Fold 2 ====================
{'end': Timestamp('2021-08-23 00:00:00', freq='7D'),
 'i_split': 2,
 'start': Timestamp('2021-08-16 00:00:00', freq='7D'),
 'test': 298878,
 'test_items': 6679,
 'test_users': 110076,
 'train': 5051815,
 'train_items': 15577,
 'train_users': 906071}


In [14]:
pivot_results = pd.DataFrame(results).drop(columns="fold").groupby(["model"], sort=False).agg(["mean", "std"])

In [15]:
mean_metric_subset = [(metric, agg) for metric, agg in pivot_results.columns if metric != 'train_time' and agg == 'mean']
mean_time_subset = [(metric, agg) for metric, agg in pivot_results.columns if metric == 'train_time' and agg == 'mean']

In [16]:
pivot_results.style\
             .highlight_min(subset=mean_metric_subset, color='lightcoral', axis=0)\
             .highlight_max(subset=mean_metric_subset, color='lightgreen', axis=0)\
             .highlight_max(subset=mean_time_subset, color='lightcoral', axis=0)\
             .highlight_min(subset=mean_time_subset, color='lightgreen', axis=0)

# Визуальный анализ

In [17]:
def visual_analysis(model, dataset, user_ids, item_data):

    if model.is_fitted == False:
        raise Exception('model is not fitted')

    # получаем рекомендации
    recos = model.recommend(
        users=user_ids,
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )
    recos.drop(columns="score", inplace=True)
    recos['is_reco'] = True

    # собираем историю просмотров
    history_view = dataset.interactions.df.query(f'user_id in {user_ids}')\
                                          .sort_values(['user_id', 'datetime'])
    ranks = history_view.sort_values('datetime').groupby(['user_id']).datetime.rank()
    history_view['rank'] = ranks
    history_view['is_reco'] = False
    history_view.drop(columns=['weight', 'datetime'], inplace=True)

    # склеиваем историю просмотров и рекомендации
    results = pd.concat([history_view, recos])

    # добавляем данные об айтемах
    num_views = interactions.df.groupby('item_id').count().user_id
    num_views.name = 'num_view'
    results = results.merge(num_views,
                            how='inner',
                            on='item_id')
    results = results.merge(item_data,
                            how='inner',
                            on='item_id')

    results.sort_values(['user_id', 'is_reco', 'rank'], inplace=True)
    results.set_index(['user_id', 'item_id'], inplace=True)

    return results

In [18]:
user_ids = [666262, 672861, 955527]
dataset = Dataset.construct(interactions.df)
item_data = items[['item_id', 'title', 'genres', 'countries']]

In [19]:
model = deepcopy(models['popular']).fit(dataset)

In [20]:
results = visual_analysis(model, dataset, user_ids, item_data)

In [21]:
results

rank  is_reco  num_view                          title  \
user_id item_id                                                           
666262  93        1.0    False         1           Дом ночных призраков   
        10440     1.0     True    202457                    Хрустальный   
        15297     2.0     True    193123                Клиника счастья   
        9728      3.0     True    132865              Гнев человеческий   
        13865     4.0     True    122119                       Девятаев   
        4151      5.0     True     91167         Секреты семейной жизни   
        3734      6.0     True     74803   Прабабушка легкого поведения   
        2657      7.0     True     68581                     Подслушано   
        4880      8.0     True     55043                          Афера   
        142       9.0     True     45367                           Маша   
        6809     10.0     True     40372                          Дуров   
672861  25        1.0    False        47  Медвежонок Винни и его друзья   
        32        2.0    False       181                 В ритме сердца   
        10440     1.0     True    202457                    Хрустальный   
        15297     2.0     True    193123                Клиника счастья   
        9728      3.0     True    132865              Гнев человеческий   
        13865     4.0     True    122119                       Девятаев   
        4151      5.0     True     91167         Секреты семейной жизни   
        3734      6.0     True     74803   Прабабушка легкого поведения   
        2657      7.0     True     68581                     Подслушано   
        4880      8.0     True     55043                          Афера   
        142       9.0     True     45367                           Маша   
        6809     10.0     True     40372                          Дуров   
955527  21        1.0    False        12                    Признание 5   
        10440     1.0     True    202457                    Хрустальный   
        15297     2.0     True    193123                Клиника счастья   
        9728      3.0     True    132865              Гнев человеческий   
        13865     4.0     True    122119                       Девятаев   
        4151      5.0     True     91167         Секреты семейной жизни   
        3734      6.0     True     74803   Прабабушка легкого поведения   
        2657      7.0     True     68581                     Подслушано   
        4880      8.0     True     55043                          Афера   
        142       9.0     True     45367                           Маша   
        6809     10.0     True     40372                          Дуров   

                                                    genres  \
user_id item_id                                              
666262  93          зарубежные, криминал, детективы, ужасы   
        10440                          триллеры, детективы   
        15297                             драмы, мелодрамы   
        9728                             боевики, триллеры   
        13865                  драмы, военные, приключения   
        4151                                       комедии   
        3734                                       комедии   
        2657                               драмы, триллеры   
        4880                                       комедии   
        142                                драмы, триллеры   
        6809                                документальное   
672861  25       мюзиклы, мультфильм, приключения, комедии   
        32                       драмы, мюзиклы, мелодрамы   
        10440                          триллеры, детективы   
        15297                             драмы, мелодрамы   
        9728                             боевики, триллеры   
        13865                  драмы, военные, приключения   
        4151                                       комедии   
        3734                                       комедии   
        2657  